<a href="https://colab.research.google.com/github/Dark-Sied/Intent_Classification/blob/master/Intent_classification_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
def load_dataset(filename):
  df = pd.read_csv(filename, encoding = "latin1", names = ["Sentence", "Intent"])
  print(df.head())
  intent = df["Intent"]
  unique_intent = list(set(intent))
  sentences = list(df["Sentence"])
  
  return (intent, unique_intent, sentences)
  


In [4]:
intent, unique_intent, sentences = load_dataset("Dataset.csv")

                Sentence          Intent
0       Need help pleese  commonQ.assist
1              Need help  commonQ.assist
2       I need some info  commonQ.assist
3      Will you help me?  commonQ.assist
4  What else can you do?  commonQ.assist


In [5]:
print(sentences[:5])

['Need help pleese', 'Need help', 'I need some info', 'Will you help me?', 'What else can you do?']


In [6]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/suraj18025/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/suraj18025/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
#define stemmer
stemmer = LancasterStemmer()

In [8]:
def cleaning(sentences):
  words = []
  for s in sentences:
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
    w = word_tokenize(clean)
    #stemming
    words.append([i.lower() for i in w])
    
  return words  

In [9]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])  


1113
[['need', 'help', 'pleese'], ['need', 'help']]


In [10]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
  token = Tokenizer(filters = filters)
  token.fit_on_texts(words)
  return token

In [11]:
def max_length(words):
  return(len(max(words, key = len)))
  

In [12]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

Vocab Size = 492 and Maximum length = 28


In [13]:
def encoding_doc(token, words):
  return(token.texts_to_sequences(words))

In [14]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [15]:
def padding_doc(encoded_doc, max_length):
  return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [16]:
padded_doc = padding_doc(encoded_doc, max_length)

In [17]:
padded_doc[:5]

array([[ 25,  77, 332,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [ 25,  77,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  1,  25, 198, 181,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [ 51,  10,  77,  16,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  8, 268,   4,  10,  30,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int32)

In [18]:
print("Shape of padded docs = ",padded_doc.shape)

Shape of padded docs =  (1113, 28)


In [19]:
#tokenizer with filter changed
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')

In [46]:
str(output_tokenizer.word_index)

"{'commonq.how': 1, 'faq.borrow_use': 2, 'commonq.wait': 3, 'faq.approval_time': 4, 'faq.aadhaar_missing': 5, 'faq.bad_service': 6, 'commonq.not_giving': 7, 'faq.apply_register': 8, 'faq.borrow_limit': 9, 'faq.biz_category_missing': 10, 'faq.biz_simpler': 11, 'commonq.just_details': 12, 'commonq.bot': 13, 'commonq.name': 14, 'commonq.query': 15, 'faq.address_proof': 16, 'contact.contact': 17, 'faq.banking_option_missing': 18, 'commonq.assist': 19, 'faq.application_process': 20, 'faq.biz_new': 21}"

In [21]:
encoded_output = encoding_doc(output_tokenizer, intent)

In [22]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [43]:
dict = {}
for i in encoded_output:
    try:
        
        dict[i[0]] +=1
    except:
        dict[i[0]] =1

In [45]:
str(dict)

'{19: 20, 13: 26, 1: 12, 12: 6, 14: 18, 7: 11, 15: 7, 3: 7, 17: 136, 5: 35, 16: 59, 20: 203, 8: 62, 4: 62, 6: 14, 18: 54, 10: 40, 21: 68, 11: 94, 9: 76, 2: 103}'

In [24]:
def one_hot(encode):
  o = OneHotEncoder(sparse = False)
  return(o.fit_transform(encode))

In [25]:
output_one_hot = one_hot(encoded_output)

/home/suraj18025/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [28]:
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)


In [29]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (890, 28) and train_Y = (890, 21)
Shape of val_X = (223, 28) and val_Y = (223, 21)


In [30]:
def create_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
  model.add(Bidirectional(LSTM(128)))
#   model.add(LSTM(128))
  model.add(Dense(32, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(21, activation = "softmax"))
  
  return model

In [31]:
model = create_model(vocab_size, max_length)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 128)           62976     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 21)                693       
Total params: 335,061
Trainable params: 272,085
Non-trainable params: 62,976
_________________________________________________________________


In [32]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit(train_X, train_Y, epochs = 100, batch_size = 32, validation_data = (val_X, val_Y), callbacks = [checkpoint])

Train on 890 samples, validate on 223 samples
Epoch 1/100
890/890 [==============================] - 3s 3ms/step - loss: 2.9605 - accuracy: 0.1449 - val_loss: 2.8635 - val_accuracy: 0.1749

Epoch 00001: val_loss improved from inf to 2.86346, saving model to model.h5
Epoch 2/100
890/890 [==============================] - 1s 1ms/step - loss: 2.8857 - accuracy: 0.1416 - val_loss: 2.8136 - val_accuracy: 0.1749

Epoch 00002: val_loss improved from 2.86346 to 2.81362, saving model to model.h5
Epoch 3/100
890/890 [==============================] - 1s 1ms/step - loss: 2.8356 - accuracy: 0.1393 - val_loss: 2.7266 - val_accuracy: 0.1749

Epoch 00003: val_loss improved from 2.81362 to 2.72657, saving model to model.h5
Epoch 4/100
890/890 [==============================] - 1s 1ms/step - loss: 2.7898 - accuracy: 0.1562 - val_loss: 2.6761 - val_accuracy: 0.1749

Epoch 00004: val_loss improved from 2.72657 to 2.67606, saving model to model.h5
Epoch 5/100
890/890 [==============================] - 1s 

In [33]:
 model = load_model("model.h5")

In [34]:
def predictions(text):
  clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
  test_word = word_tokenize(clean)
  test_word = [w.lower() for w in test_word]
  test_ls = word_tokenizer.texts_to_sequences(test_word)
  print(test_word)
  #Check for unknown words
  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
  x = padding_doc(test_ls, max_length)
  
  pred = model.predict_proba(x)
  
  
  return pred


  

In [35]:
def get_final_output(pred, classes):
  predictions = pred[0]
 
  classes = np.array(classes)
  ids = np.argsort(-predictions)
  classes = classes[ids]
  predictions = -np.sort(-predictions)
 
  for i in range(pred.shape[1]):
    print("%s has confidence = %s" % (classes[i], (predictions[i])))



In [36]:
text = "Can you help me?"
pred = predictions(text)
get_final_output(pred, unique_intent)

['can', 'you', 'help', 'me']
commonQ.bot has confidence = 0.1766537
faq.bad_service has confidence = 0.11811465
commonQ.query has confidence = 0.09906488
commonQ.assist has confidence = 0.097041965
faq.biz_new has confidence = 0.077170715
contact.contact has confidence = 0.07646705
commonQ.not_giving has confidence = 0.06568884
commonQ.name has confidence = 0.053662248
commonQ.how has confidence = 0.035175517
faq.aadhaar_missing has confidence = 0.033894695
commonQ.just_details has confidence = 0.030430667
commonQ.wait has confidence = 0.030384624
faq.biz_simpler has confidence = 0.029610055
faq.banking_option_missing has confidence = 0.020859007
faq.borrow_limit has confidence = 0.017620571
faq.apply_register has confidence = 0.011174968
faq.approval_time has confidence = 0.011127081
faq.borrow_use has confidence = 0.0055858484
faq.biz_category_missing has confidence = 0.0046971096
faq.application_process has confidence = 0.004112841
faq.address_proof has confidence = 0.0014629527
